In [1]:
import spacy
import random
from spacy.training import Example
from spacy.util import minibatch, compounding

# Cargar el modelo base
nlp = spacy.load("en_core_web_sm")

# Añadir la etiqueta si no existe en el modelo
ner = nlp.get_pipe("ner")
ner.add_label("DINERO")  # Ejemplo de una nueva etiqueta

# Datos de entrenamiento
TRAIN_DATA = [
    ("Apple is looking at buying U.K. startup for $1 billion", {
        "entities": [(0, 5, "ORG"), (27, 31, "GPE"), (44, 52, "DINERO")]
    }),
    ("San Francisco considers banning sidewalk delivery robots", {
        "entities": [(0, 13, "GPE")]
    }),
    # Más ejemplos...
]

# Desactivar otros componentes del pipeline
pipe_exceptions = ["ner"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# Entrenar el modelo
with nlp.disable_pipes(*unaffected_pipes):  # Solo entrenar NER
    optimizer = nlp.resume_training()
    for itn in range(100):  # Número de iteraciones
        random.shuffle(TRAIN_DATA)
        losses = {}
        # Lote a lote (minibatch)
        batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            for text, annotations in batch:
                # Crear un objeto Example
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                nlp.update([example], sgd=optimizer, drop=0.35, losses=losses)
        print("Losses", losses)

# Guardar el modelo entrenado
nlp.to_disk("path_to_save_model")


c:\Users\lol_a\AppData\Local\Programs\Python\Python39\lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Apple is looking at buying U.K. startup for $1 bil..." with entities "[(0, 5, 'ORG'), (27, 31, 'GPE'), (44, 52, 'DINERO'...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(


Losses {'ner': 0.048798730614108166}
Losses {'ner': 0.0012802985675565751}
Losses {'ner': 0.0012185646629109567}
Losses {'ner': 8.157046629099546e-07}
Losses {'ner': 0.00019530642358557673}
Losses {'ner': 4.450582334967897e-05}
Losses {'ner': 0.18067293079781294}
Losses {'ner': 0.14045198610014878}
Losses {'ner': 4.890000652918326e-06}
Losses {'ner': 3.638184456769861e-06}
Losses {'ner': 2.0545707631427339e-07}
Losses {'ner': 4.450217688959262e-07}
Losses {'ner': 0.022578261460264357}
Losses {'ner': 2.6598637096896998e-08}
Losses {'ner': 2.61504306772052e-09}
Losses {'ner': 0.0011053132880319335}
Losses {'ner': 4.216021439873095e-09}
Losses {'ner': 7.328441062222972e-08}
Losses {'ner': 2.001467420061342e-07}
Losses {'ner': 8.343289376522011e-08}
Losses {'ner': 1.2146562746976504e-07}
Losses {'ner': 5.874626488266721e-07}
Losses {'ner': 5.27352229570635e-10}
Losses {'ner': 5.59730097535646e-08}
Losses {'ner': 0.004377023872811205}
Losses {'ner': 6.8776953653842716e-09}
Losses {'ner': 9.

In [2]:
# Cargar el modelo desde el disco
nlp = spacy.load("path_to_save_model")

In [3]:
# Ejemplo de texto para probar el modelo
test_text = "Apple is buying a startup in London for $2 billion"

# Procesar el texto con el modelo
doc = nlp(test_text)

# Mostrar las entidades detectadas
for ent in doc.ents:
    print(ent.text, ent.label_)

Apple ORG
London GPE
$2 billion MONEY


In [18]:
def etiquetar_tweet(tweet, calles, colonias):
    entities = []
    for calle in calles:
        start = tweet.find(calle)
        if start != -1:
            end = start + len(calle)
            entities.append((start, end, "CALLE"))
    
    for colonia in colonias:
        start = tweet.find(colonia)
        if start != -1:
            end = start + len(colonia)
            entities.append((start, end, "COLONIA"))

    return (tweet, {"entities": entities})

# Ejemplo de uso
calles = ["Insurgentes", "Reforma", "Chapultepec"]
colonias = ["Roma", "Condesa", "Polanco"]
tweet = "Vivo en la calle Reforma en la colonia Condesa"

training_data = [etiquetar_tweet(tweet, calles, colonias)]

In [19]:
training_data

[('Vivo en la calle Reforma en la colonia Condesa',
  {'entities': [(17, 24, 'CALLE'), (39, 46, 'COLONIA')]})]

In [7]:
from fuzzywuzzy import process
from transformers import BertTokenizer

# Función para tokenizar los textos
def tokenize_function(text):
    return tokenizer.tokenize(text)

# Inicializar el tokenizador BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Palabra objetivo (puede ser más de una palabra)
target_word = "santa lucia"

# Texto del que quieres extraer y comparar palabras
text = "Estamos celebrando la festividad de snta luci, pero algunos dicen que es o sant lucia."

# Tokenizar el texto en palabras
tokens = tokenize_function(text)

# Convertir los tokens nuevamente a texto para compararlos
tokenized_words = tokenizer.convert_tokens_to_string(tokens).split()

# Dividir la palabra objetivo en partes
target_tokens = target_word.split()

# Buscar coincidencias para cada token de la palabra objetivo
matches = []
for target_token in target_tokens:
    best_match = process.extractOne(target_token, tokenized_words)
    matches.append(best_match)

# Calcular la puntuación promedio de las coincidencias
average_score = sum(match[1] for match in matches) / len(matches)

# Encontrar las palabras originales en el texto
original_words = []
for match in matches:
    best_match_token = match[0]
    for word in text.split():
        if best_match_token in word:
            original_words.append(word)
            break

# Unir las palabras originales para obtener la frase final
matched_phrase = ' '.join(original_words)

print(f"La frase más cercana encontrada es: '{matched_phrase}' con una puntuación promedio de {average_score:.2f}")


La frase más cercana encontrada es: 'snta lucia.' con una puntuación promedio de 94.50


In [6]:
from fuzzywuzzy import process
from transformers import BertTokenizer

# Función para tokenizar los textos
def tokenize_function(text):
    return tokenizer.tokenize(text)

# Inicializar el tokenizador BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Palabra objetivo (puede ser más de una palabra)
target_word = "santa lucia"

# Texto del que quieres extraer y comparar palabras
text = "Estamos celebrando la festividad de snta luci, pero algunos dicen que es o sant lucia."

# Tokenizar el texto en palabras
tokens = tokenize_function(text)

# Convertir los tokens nuevamente a texto para compararlos
tokenized_words = tokenizer.convert_tokens_to_string(tokens).split()

# Dividir la palabra objetivo en partes
target_tokens = target_word.split()

# Buscar coincidencia para la primera token
first_match = process.extractOne(target_tokens[0], tokenized_words)
best_match_token1 = first_match[0]
position1 = tokenized_words.index(best_match_token1)

# Encontrar las palabras originales en el texto para la primera token
original_word1 = ""
for word in text.split():
    if best_match_token1 in word:
        original_word1 = word
        break

# Buscar coincidencia para la segunda token cercana a la primera
second_match = None
for match in process.extract(target_tokens[1], tokenized_words, limit=5):
    best_match_token2 = match[0]
    position2 = tokenized_words.index(best_match_token2)
    if abs(position2 - position1) <= len(target_word.split()):
        second_match = match
        break

# Si no se encuentra una coincidencia cercana para la segunda token
if second_match is None:
    # Buscar la mejor coincidencia sin importar la proximidad
    second_match = process.extractOne(target_tokens[1], tokenized_words)
    best_match_token2 = second_match[0]
    position2 = tokenized_words.index(best_match_token2)

# Encontrar las palabras originales en el texto para la segunda token
original_word2 = ""
for word in text.split():
    if best_match_token2 in word:
        original_word2 = word
        break

# Unir las palabras originales para obtener la frase final
matched_phrase = f"{original_word1} {original_word2}"
average_score = (first_match[1] + second_match[1]) / 2

print(f"La frase más cercana encontrada es: '{matched_phrase}' con una puntuación promedio de {average_score:.2f}")


La frase más cercana encontrada es: 'snta luci,' con una puntuación promedio de 89.00


In [2]:
from fuzzywuzzy import process
from transformers import BertTokenizer

# Función para tokenizar los textos
def tokenize_function(text):
    return tokenizer.tokenize(text)

# Inicializar el tokenizador BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Palabra objetivo (puede ser más de una palabra)
target_word = "San Pedro Garza García"

# Texto del que quieres extraer y comparar palabras
text = "Voy a ir a San P al otro lado de la loma, mañana"

# Tokenizar el texto en palabras
tokens = tokenize_function(text)

# Convertir los tokens nuevamente a texto para compararlos
tokenized_words = tokenizer.convert_tokens_to_string(tokens).split()

# Dividir la palabra objetivo en partes
target_tokens = target_word.split()

# Verificar si el texto tiene suficientes palabras tokenizadas
if len(tokenized_words) < len(target_tokens):
    print("El texto no tiene suficientes palabras para comparar con la palabra objetivo.")
else:
    # Buscar coincidencia para la primera token
    first_match = process.extractOne(target_tokens[0], tokenized_words)
    best_match_token1 = first_match[0]
    position1 = tokenized_words.index(best_match_token1)

    # Encontrar las palabras originales en el texto para la primera token
    original_word1 = ""
    for word in text.split():
        if best_match_token1.lower() in word.lower():
            original_word1 = word
            break

    # Buscar coincidencia para la segunda token cercana a la primera (si existe)
    if len(target_tokens) > 1:
        second_match = None
        for match in process.extract(target_tokens[1], tokenized_words, limit=5):
            best_match_token2 = match[0]
            position2 = tokenized_words.index(best_match_token2)
            if abs(position2 - position1) <= len(target_word.split()):
                second_match = match
                break

        # Si no se encuentra una coincidencia cercana para la segunda token
        if second_match is None:
            # Buscar la mejor coincidencia sin importar la proximidad
            second_match = process.extractOne(target_tokens[1], tokenized_words)
            best_match_token2 = second_match[0]
            position2 = tokenized_words.index(best_match_token2)

        # Encontrar las palabras originales en el texto para la segunda token
        original_word2 = ""
        for word in text.split():
            if best_match_token2.lower() in word.lower():
                original_word2 = word
                break

        # Unir las palabras originales para obtener la frase final
        matched_phrase = f"{original_word1} {original_word2}".strip()
        average_score = (first_match[1] + second_match[1]) / 2
    else:
        matched_phrase = original_word1
        average_score = first_match[1]

    if matched_phrase:
        print(f"La frase más cercana encontrada es: '{matched_phrase}' con una puntuación promedio de {average_score:.2f}")
    else:
        print("No se encontró una coincidencia adecuada en el texto.")


La frase más cercana encontrada es: 'San P' con una puntuación promedio de 95.00


In [1]:
import pandas as pd
import unicodedata
import re

def remove_accents_and_punctuation(text):
    # Eliminar acentos
    text = ''.join(
        (c for c in unicodedata.normalize('NFD', text)
         if unicodedata.category(c) != 'Mn')
    )
    # Eliminar signos de puntuación
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Aplicar la función a todo el DataFrame
df1 = pd.read_csv('tweets_classified.csv')
df2 = pd.read_csv('calles_colonias_monterrey.csv')

tweets_df = df1.applymap(lambda x: remove_accents_and_punctuation(x) if isinstance(x, str) else x)
calles_df = df2.applymap(lambda x: remove_accents_and_punctuation(x) if isinstance(x, str) else x)



In [2]:
import pandas as pd
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
from transformers import BertTokenizer

# Inicializar el tokenizador BERT
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Cargar los datos desde los archivos CSV
#calles_df = pd.read_csv('calles_colonias_monterrey.csv')
#tweets_df = pd.read_csv('tweets_classified.csv')

# Definir el umbral de similitud
similarity_threshold = 80  # Ajusta este valor según lo que consideres adecuado

# Iterar sobre cada tweet
for tweet_index, tweet_row in tweets_df.iterrows():
    text = tweet_row['Texto']

    # Tokenizar el texto del tweet
    tokens = tokenizer.tokenize(text)
    tokenized_words = tokenizer.convert_tokens_to_string(tokens).split()

    best_overall_match = None
    best_matched_phrase = ""
    best_target_word = ""

    # Iterar sobre cada palabra objetivo
    for _, calle_row in calles_df.iterrows():
        target_word = calle_row['name']
        target_tokens = target_word.split()

        if len(tokenized_words) < len(target_tokens):
            continue  # Saltar si el tweet no tiene suficientes palabras tokenizadas

        # Buscar coincidencia para la primera token
        first_match = process.extractOne(target_tokens[0], tokenized_words)
        best_match_token1 = first_match[0]
        position1 = tokenized_words.index(best_match_token1)

        # Encontrar la palabra original en el texto para la primera token
        original_word1 = ""
        for word in text.split():
            if best_match_token1.lower() in word.lower():
                original_word1 = word
                break

        # Buscar coincidencia para la segunda token cercana a la primera (si existe)
        if len(target_tokens) > 1:
            second_match = None
            for match in process.extract(target_tokens[1], tokenized_words, limit=5):
                best_match_token2 = match[0]
                position2 = tokenized_words.index(best_match_token2)
                if abs(position2 - position1) <= len(target_word.split()):
                    second_match = match
                    break

            # Si no se encuentra una coincidencia cercana para la segunda token
            if second_match is None:
                # Buscar la mejor coincidencia sin importar la proximidad
                second_match = process.extractOne(target_tokens[1], tokenized_words)
                best_match_token2 = second_match[0]
                position2 = tokenized_words.index(best_match_token2)

            # Encontrar las palabras originales en el texto para la segunda token
            original_word2 = ""
            for word in text.split():
                if best_match_token2.lower() in word.lower():
                    original_word2 = word
                    break

            # Unir las palabras originales para obtener la frase final
            matched_phrase = f"{original_word1} {original_word2}".strip()
            average_score = (first_match[1] + second_match[1]) / 2
        else:
            matched_phrase = original_word1
            average_score = first_match[1]

        # Evitar coincidencias de frases que no tienen sentido (ej. "del el")
        if any(word not in target_word.lower() for word in matched_phrase.lower().split()):
            continue

        # Comparar la coincidencia encontrada con la palabra objetivo usando FuzzyWuzzy
        similarity_score = fuzz.ratio(matched_phrase.lower(), target_word.lower())

        # Verificar si la similitud es mayor que el umbral definido
        if similarity_score >= similarity_threshold:
            # Comparar con la mejor coincidencia encontrada hasta ahora
            if best_overall_match is None or average_score > best_overall_match:
                best_overall_match = average_score
                best_matched_phrase = matched_phrase
                best_target_word = target_word

    if best_matched_phrase:
        print(f"Tweet {tweet_index}: '{text}'")
        print(f"Mejor coincidencia: '{best_matched_phrase}' usando '{best_target_word}' con una puntuación promedio de {best_overall_match:.2f}\n")
    else:
        print(f"Tweet {tweet_index}: No se encontró una coincidencia adecuada.\n")


c:\Users\lol_a\AppData\Local\Programs\Python\Python39\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
c:\Users\lol_a\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Tweet 0: 'cuidadodelagua fuga de agua limpia en un domicilio en olmos entre av potrero de anahuac y encinos col potrero anahuac sannicolas via enriquemgo cc aydmonterrey kmgn'
Mejor coincidencia: 'anahuac' usando 'Anahuac' con una puntuación promedio de 100.00

Tweet 1: No se encontró una coincidencia adecuada.

Tweet 2: 'luego de las lluvias torrenciales registradas en campeche se apoya a la poblacion de las zonas bajas inundadas ademas se bombea agua sobre la autopista champotoncampeche en el cruce del arroyo siho'
Mejor coincidencia: 'del el' usando 'Del Reloj' con una puntuación promedio de 95.00

Tweet 3: 'un canal de bajapresion en la sierra madre occidental y centro de provocara lluvias torrenciales en gro oax pue y ver intensas en camp tamps chis col hgo jal mich qroo slp tab y yuc muy fuertes en edomex gto mor nay nl qro y tlax'
Mejor coincidencia: 'sierra en' usando 'Sierra Morena' con una puntuación promedio de 95.00

Tweet 4: 'se preven lluvias puntuales torrenciales en gue

KeyboardInterrupt: 